<a href="https://colab.research.google.com/github/sun0809/BERT-project/blob/main/Inference_doc_cls_deploy_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.



In [1]:
!pip install ratsnlp

     |████████████████████████████████| 42 kB 485 kB/s 
     |████████████████████████████████| 806 kB 7.0 MB/s 
     |████████████████████████████████| 2.8 MB 21.6 MB/s 
     |████████████████████████████████| 57 kB 4.0 MB/s 
     |████████████████████████████████| 829 kB 57.2 MB/s 
     |████████████████████████████████| 133 kB 54.3 MB/s 
     |████████████████████████████████| 636 kB 54.4 MB/s 
     |████████████████████████████████| 397 kB 22.5 MB/s 
     |████████████████████████████████| 67 kB 2.8 MB/s 
     |████████████████████████████████| 895 kB 53.2 MB/s 
     |████████████████████████████████| 3.3 MB 17.4 MB/s 
     |████████████████████████████████| 1.1 MB 39.7 MB/s 
     |████████████████████████████████| 96 kB 4.7 MB/s 
     |████████████████████████████████| 271 kB 59.7 MB/s 
     |████████████████████████████████| 94 kB 2.7 MB/s 
     |████████████████████████████████| 144 kB 61.1 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha

# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [3]:
#인퍼런스 설정

from ratsnlp.nlpbook.classification import ClassificationDeployArguments
args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",  # 프리트레인 모델 
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-doccls",  # 모델 저장 위치
    max_seq_length=128,  # 입력 문장 최대 길이
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-doccls/epoch=0-val_loss=0.27.ckpt


# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

In [4]:
# 토크나이저 초기화하기
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

In [5]:
import torch
from transformers import BertConfig, BertForSequenceClassification

#파인튜닝한 모델의 체크포인트 로드
fine_tuned_model_ckpt = torch.load(   
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)
# 앞 절 파인튜닝 때 사용한 pretrained_model_name에 해당하는 모델의 설정값들을 읽어들입니다. 
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)
#해당 설정값대로 모델을 설정합니다. 
model = BertForSequenceClassification(pretrained_model_config)

#초기화한 bert 모델에 체크포인트(fine_tuned_model_ckpt)를 주입합니다. 
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

#모델이 평가모드로 전환됩니다.  evaluation 과정에서 사용하지 않아야 하는 layer들을 알아서 off 시키도록 하는 함수입니다. drop-out 등 학습 때만 사용하는 기법들을 뮤효화 합니다.
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

# 인퍼런스 함수 선언
# 모델 출력값 만들고 후처리하기

인퍼런스 함수를 선언합니다.

In [6]:
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})  #inputs를 파이토치 텐서로 바꾼후 모델 계산하기
        prob = outputs.logits.softmax(dim=1)  ## outputs.logit은 soft-max함수에 넣기 전 로짓(logit)형태이다. 
        positive_prob = round(prob[0][1].item(), 4) ## 긍정/부정일 확률을 소수점 4자리로 반올림
        negative_prob = round(prob[0][0].item(), 4)
        pred = "긍정 (positive)" if torch.argmax(prob) == 1 else "부정 (negative)"  # 예측확률의 최댓값 위치에 따라 pred만들기
    return {
        'sentence': sentence,
        'prediction': pred,
        'positive_data': f"긍정 {positive_prob}",
        'negative_data': f"부정 {negative_prob}",
        'positive_width': f"{positive_prob * 100}%",
        'negative_width': f"{negative_prob * 100}%",
    }

# 웹서비스 만들기 준비

`ngrok`은 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구입니다. `ngrok`을 실행하려면 [회원가입](https://dashboard.ngrok.com/signup) 후 [로그인](https://dashboard.ngrok.com/login)을 한 뒤 [이곳](https://dashboard.ngrok.com/get-started/your-authtoken)에 접속해 인증 토큰(authtoken)을 확인해야 합니다. 예를 들어 확인된 `authtoken`이 `test111`이라면 다음과 같이 실행합니다.

```bash
!mkdir /root/.ngrok2 && echo "authtoken: test111" > /root/.ngrok2/ngrok.yml
```

In [8]:
!mkdir /root/.ngrok2 && echo "authtoken: 25PKt5xXqtDZ7pBseBGD893GHHL_2fe42kHi1Nh4cyJvRcd2f" > /root/.ngrok2/ngrok.yml


mkdir: cannot create directory ‘/root/.ngrok2’: File exists


# 웹서비스 개시
아래처럼 실행해 인퍼런스 함수를 웹서비스로 만듭니다.

In [10]:
from ratsnlp.nlpbook.classification import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.classification.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://b6e1-35-201-247-81.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [21/Feb/2022 06:36:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Feb/2022 06:36:22] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [21/Feb/2022 06:36:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Feb/2022 06:36:26] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [21/Feb/2022 06:36:30] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [21/Feb/2022 06:36:34] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [21/Feb/2022 06:36:37] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [21/Feb/2022 06:36:41] "POST /api HTTP/1.1" 200 -
